In [1]:
#set the API key
from tmdbv3api import TMDb
from tmdbv3api import Movie
from bs4 import BeautifulSoup
tmdb = TMDb()
movie = Movie()
tmdb.api_key = 'c6cd98753f10dc390a8c4a5b4b83d47c'
tmdb.language = 'en'
tmdb.debug = True

### Download the webpage using “requests”

In [2]:
import requests


In [3]:
needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
response = requests.get("https://www.themoviedb.org/tv", headers = needed_headers)


In [4]:
response.status_code
##succes between 200-299

200

In [5]:
dwn_content = response.text
len(dwn_content)

193920

In [6]:
dwn_content[:500]

'<!DOCTYPE html>\n<html lang="en" class="no-js">\n  <head>\n    <title>Popular TV Shows &#8212; The Movie Database (TMDB)</title>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n    <meta http-equiv="cleartype" content="on">\n    <meta charset="utf-8">\n    <meta name="keywords" content="Movies, TV Shows, Streaming, Reviews, API, Actors, Actresses, Photos, User Ratings, Synopsis, Trailers, Teasers, Credits, Cast">\n    <meta name="mobile-web-app-capable" content="yes">\n    <meta name="apple'

### Parse the HTML source code using beautiful soup.

##### To parse the HTML source code we are going to use BeautifulSoup() function that takes two arguments.

###### content of the page.
###### ‘html.parser’

In [7]:
test_doc = BeautifulSoup(response.text, 'html.parser')

In [8]:
type(test_doc)

bs4.BeautifulSoup

In [9]:
test_doc.find('title')

<title>Popular TV Shows — The Movie Database (TMDB)</title>

In [10]:
#download a webpage
def get_page_content(url):
    # In this case , we are going to give request.get function headers to avoid the Status code Error 403

    get_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response_page = requests.get(url, headers = get_headers )
    # we are going to raise exception here if status code gives any value other than 200.
    if not response_page.ok:
        raise Exception ("Failed to request the data. Status Code:- {}".format(response_page.status_code))
    else:
        page_content = response_page.text
        doc_page = BeautifulSoup(page_content, "html.parser")
        return doc_page

In [11]:
# execution of the function
popular_shows_url = "https://www.themoviedb.org/tv"
doc = get_page_content(popular_shows_url)

In [12]:
#let's try to get the title of the page to check if our function works. 

doc.title.text

'Popular TV Shows — The Movie Database (TMDB)'

### Extract all the data from the webpages.

##### example 

In [13]:
# Now that we know the class let's trty to get the title of the first movie. 

doc.find_all('div', {'class': 'card style_1'})[0].h2.text

'Wednesday'

In [14]:
doc.find_all('div', {'class': 'user_score_chart'})[0]['data-percent']

'88.0'

In [16]:
def empty_dict():
    scraped_dict = {  
                    'Title': [],
                    'User_rating': [], 
                    'Release_date':[], 
                    'Current_season': [],
                    'Current_season_Episodes': [], 
                    'Tagline': [],
                    'Genre': [],
                    'Cast': []   
                    }
    return scraped_dict

#### function for the rating

In [17]:
def user_score_info(tag_user_score, i, scraped_dict):
    if tag_user_score[i]['data-percent'] == '0':
        scraped_dict['User_rating'].append('Not rated yet')
    else:
        scraped_dict['User_rating'].append(tag_user_score[i]['data-percent'])

In [18]:
doc.find_all('div', {'class': 'card style_1'})[0].h2.a['href']

'/tv/119051'

In [19]:
def get_show_info(doc_page):
    base_link_1 = "https://www.themoviedb.org"
    tag_title = tag_premired_date = tag_shows_page = doc_page.find_all('div', {'class': 'card style_1'})
    tag_user_score = doc_page.find_all('div', {"user_score_chart"}) 
    
    doc_2_list = []
    for link in tag_shows_page:
        # here we are creating the list of all the individual pages of the shows which will come handy in other functions. 
        doc_2_list.append(get_page_content("https://www.themoviedb.org" + link.h2.a['href']))
       # we are going to have the function to return the list of all the information as elements. 
    return tag_title, tag_user_score, doc_2_list

In [20]:
# lets see if the function returns the list of the information we tried to get earlier. 
len(get_show_info(doc))

3

In [21]:
doc_2 = get_page_content("https://www.themoviedb.org/tv/91363")

In [22]:
tag_genre = doc_2.find('span', {"class": "genres"})
tag_genre_list = tag_genre.find_all('a')

check_genre =[]
for tag in tag_genre_list:
    check_genre.append(tag.text)

check_genre

['Animation', 'Action & Adventure', 'Sci-Fi & Fantasy']

##### genres

In [24]:
# lets create a function to get the genres for the show. 
# i here denotes the element of the list vairable ``doc2_page`` that contains different doc pages. Will come handy later on.
def get_genres(doc2_page, i):
    genres_tags = doc2_page[i].find('span', {"class": "genres"}).find_all('a')
    check_genre =[]
    
    for tag in genres_tags:
        check_genre.append(tag.text)
    return check_genre

##### tagline

In [25]:
tag_tagline = doc_2.find('h3',{"class": 'tagline'})

def tagline_info(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('h3',{"class": 'tagline'}):
        scraped_dict['Tagline'].append(doc_2_list[i].find('h3',{"class": 'tagline'}).text)
    else:
        scraped_dict['Tagline'].append("No Tagline")

##### cast

# i here denotes the the element of the list type variable``doc2_page`` that contains different doc pages.

def get_show_cast(doc2_page, i):
    cast_tags = doc2_page[i].find_all('li', {'class': 'card'})
    cast_lis = []
    
    for t in cast_tags:
         cast_lis.append(t.p.text)
    
    return cast_lis

##### current season and episodes

In [28]:
tag_episodes = doc_2.find_all('div' , {'class': 'flex'})
# extracing current season from h2 tag under class flex.
tag_episodes[1].h2.text 

'Season 1'

In [29]:
print('2021 | 9 Episodes'[7:])

tag_episodes[1].h4.text[7:]

9 Episodes


'9 Episodes'

### Compile extracted information into python lists and dictionaries.

In [30]:
import pandas as pd

def get_show_details(t_title, t_user_score, docs_2_list):
    # excuting a function here that empties the dictionary every time the function is called.
    scraped_dict =  empty_dict()
    for i in range (0, len(t_title)):
        scraped_dict['Title'].append(t_title[i].h2.text)
        user_score_info(t_user_score, i, scraped_dict)    
        scraped_dict['Release_date'].append(t_title[i].p.text)
        scraped_dict['Current_season'].append(docs_2_list[i].find_all('div' , {'class': 'flex'})[1].h2.text)
        scraped_dict['Current_season_Episodes'].append(docs_2_list[i].find_all('div' , {'class': 'flex'})[1].h4.text[7:])
        tagline_info(docs_2_list, i, scraped_dict)  
        scraped_dict['Genre'].append(get_genres(docs_2_list, i))        
        scraped_dict['Cast'].append(get_show_cast(docs_2_list, i))
        
    return pd.DataFrame(scraped_dict)

In [31]:
# Let's excute our function to check if it works. We are going to take a look the data of dataframe.

x = get_show_details(tag_title_, tag_user_score_, doc_2_list_)
x.to_csv('check.csv')
pd.read_csv('check.csv',index_col=[0])

NameError: name 'tag_title_' is not defined

### Save the extracted information to a CSV file.


In [32]:
import os
base_link = "https://www.themoviedb.org/tv"

# 'i' here means the number of page we want to extract
def create_page_df( i, dataframe_list):
    os.makedirs('shows-data', exist_ok = True)
    next_url = base_link + '?page={}'.format(i)
    doc_top = get_page_content(next_url)
    name_tag, viewer_score_tag, doc_2_lis = get_show_info(doc_top)
    print('scraping page {} :- {}'.format(i, next_url))
    dataframe_data = get_show_details(name_tag, viewer_score_tag, doc_2_lis)
    dataframe_data.to_csv("shows-data/shows-page-{}.csv".format(i) , index = None)
    print(" ---> a CSV file with name shows-page-{}.csv has been created".format(i))
    dataframe_list.append(dataframe_data)

In [33]:
test_list = []
create_page_df(9 , test_list)

scraping page 9 :- https://www.themoviedb.org/tv?page=9
 ---> a CSV file with name shows-page-9.csv has been created


In [34]:
import pandas as pd
base_link = "https://www.themoviedb.org/tv"

def scrape_top_200_shows(base_link):
    dataframe_list = []
    # we are going to keep range up to 11 because we just need up to 200 TV shows for now. 
    for i in range(1,11):
        create_page_df(i, dataframe_list)
    # here we are using concat function so that we can merge the each dataframe that we got from the each page.    
    total_dataframe = pd.concat(dataframe_list, ignore_index = True)
    
    # with the simple command of to_csv() we can create a csv file of all the pages we extracted.
    csv_complete =  total_dataframe.to_csv('shows-data/Total-dataframe.csv', index= None)
    print(" \n a CSV file named Total-dataframe.csv with all the scraped shows has been created")

In [35]:
scrape_top_200_shows(base_link)

scraping page 1 :- https://www.themoviedb.org/tv?page=1
 ---> a CSV file with name shows-page-1.csv has been created
scraping page 2 :- https://www.themoviedb.org/tv?page=2
 ---> a CSV file with name shows-page-2.csv has been created
scraping page 3 :- https://www.themoviedb.org/tv?page=3
 ---> a CSV file with name shows-page-3.csv has been created
scraping page 4 :- https://www.themoviedb.org/tv?page=4
 ---> a CSV file with name shows-page-4.csv has been created
scraping page 5 :- https://www.themoviedb.org/tv?page=5
 ---> a CSV file with name shows-page-5.csv has been created
scraping page 6 :- https://www.themoviedb.org/tv?page=6
 ---> a CSV file with name shows-page-6.csv has been created
scraping page 7 :- https://www.themoviedb.org/tv?page=7
 ---> a CSV file with name shows-page-7.csv has been created
scraping page 8 :- https://www.themoviedb.org/tv?page=8
 ---> a CSV file with name shows-page-8.csv has been created
scraping page 9 :- https://www.themoviedb.org/tv?page=9
 ---> a 

In [36]:
pd.read_csv('shows-data/Total-dataframe.csv')[0:50]

,Title,User_rating,Release_date,Current_season,Current_season_Episodes,Tagline,Genre,Cast
0,Wednesday,88.0,"Nov 23, 2022",Season 1,8 Episodes,No Tagline,"['Sci-Fi & Fantasy', 'Mystery', 'Comedy']","['Jenna Ortega', 'Emma Myers', 'Joy Sunday', '..."
1,The Street,11.0,"Sep 05, 2005",Season 18,47 Episodes,No Tagline,"['Drama', 'Comedy']","['Michaela Badinková', 'Tereza Brodská', 'Patr..."
2,Ghum Hai Kisi Ke Pyaar Mein,54.0,"Oct 05, 2020",Season 1,711 Episodes,No Tagline,"['Drama', 'Soap', 'Family']","['Neil Bhatt', 'Siddharth Bodke', 'Shailesh Da..."
3,Lisa,63.0,"Jan 25, 2021",Season 4,87 Episodes,No Tagline,"['Drama', 'Family']","['Tinne Oltmans', 'Anouck Luyten', 'Oscar Will..."
4,Bhagya Lakshmi,46.0,"Aug 03, 2021",Season 1,439 Episodes,No Tagline,['Soap'],[]
5,Kronprinsen som försvann,20,"Dec 01, 2022",Season 1,24 Episodes,No Tagline,"['Family', 'Mystery', 'Drama']","['Lennart Jähkel', 'Dilan Gwyn', 'Xavier Canca..."
6,Mar do Sertão,43.0,"Aug 22, 2022",Season 1,108 Episodes,No Tagline,"['Soap', 'Drama']","['Sérgio Guizé', 'Isadora Cruz', 'Renato Góes'..."
7,¿Cómo se llama esta relación?,Not rated yet,"Oct 27, 2021",Season 1,381 Episodes,No Tagline,[],[]
8,Cara e Coragem,58.0,"May 30, 2022",Season 1,197 Episodes,No Tagline,"['Comedy', 'Action & Adventure', 'Soap']","['Paolla Oliveira', 'Marcelo Serrado', 'Taís A..."
9,Euronews Hoy,40,"Jan 28, 2019",Season 4,240 Episodes,No Tagline,['News'],[]
